In [2]:
!nvidia-smi
from tensorflow.keras import layers
import tensorflow as tf

Tue Mar  5 06:03:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0              27W /  70W |      2MiB / 15360MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

2024-03-05 06:03:59.061841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 06:03:59.184332: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-03-05 06:03:59.184447: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-03-05 06:03:59.184657: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-0

In [3]:
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

In [4]:
#!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

In [5]:
#unzip_data('nlp_getting_started.zip')

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state = 42)

In [8]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
len(train_df),len(test_df)

(7613, 3263)

In [12]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target : {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text : \n{text}\n")
    print("----\n")
    


Target : 0 (not real disaster)
Text : 
BESTSELLER! Fossil Dawson Mini Cross Body Bag EsÛ_ $98.00 http://t.co/HhHvKxFAIH

----

Target : 1 (real disaster)
Text : 
Here's what may have caused a Metro train to derail in downtown D.C. http://t.co/mEiSNKv5Tb

----

Target : 0 (not real disaster)
Text : 
i'd still be team usagi even if she blew up the entire solar system by some airhead misstep

----

Target : 0 (not real disaster)
Text : 
One Direction Is my pick for http://t.co/q2eBlOKeVE Fan Army #Directioners http://t.co/eNCmhz6y34 x1411

----

Target : 1 (real disaster)
Text : 
#News Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires'... http://t.co/zfYqSAwvrk

----



In [13]:
#007 Splitting data into training and validation sets.mp4
from sklearn.model_selection import train_test_split
train_sentences , val_sentences ,train_labels,val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)
                                                                            
                                                                            

In [14]:
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [15]:
#008 Converting text data to numbers using tokenisation and embeddings (overview).mp4
#009 Setting up a TensorFlow TextVectorization layer to convert text to numbers.mp4
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#OOV  = "Out of Vocabulary"
text_vectorizer = TextVectorization(max_tokens=10000,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)
                                    
                                    
                                    

2024-03-05 06:04:05.707288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-05 06:04:05.715298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-05 06:04:05.717161: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-05 06:04:05.719476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebu

In [16]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length,
                                   )

In [18]:
#010 Mapping the TextVectorization layer to text data and turning it into numbers
# Fit the text vectorizer to training text
text_vectorizer.adapt(train_sentences)

In [19]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Elecman could electrocute me and I'd say thanks.\n
Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1, 137, 628,  31,   7, 508, 132, 565,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
#[UNK] meand unknown token

In [21]:
len(words_in_vocab)

10000

In [22]:
#011 Creating an Embedding layer to turn tokenised text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length = max_length
                             )

In [23]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Maryland mansion fire that killed 6 caused by damaged plug under Christmas tree report says - Into the flames... http://t.co/P1WmkXA9d8\n
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01274914,  0.03239167, -0.01391454, ...,  0.03586625,
          0.00179486, -0.03014497],
        [ 0.03581083, -0.03858008,  0.00171137, ..., -0.04975555,
         -0.03381039, -0.00454777],
        [ 0.03860335,  0.04405329, -0.00715002, ..., -0.01335409,
          0.04424694,  0.03053036],
        ...,
        [-0.01945105, -0.00222393,  0.03193552, ...,  0.00292265,
         -0.04618113, -0.03650662],
        [ 0.04276964,  0.01833848, -0.0062881 , ..., -0.04744362,
         -0.00649086, -0.03677864],
        [-0.02656507,  0.04635983,  0.00523766, ..., -0.02776767,
          0.00641657, -0.00226146]]], dtype=float32)>

In [24]:
#check out a single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01274914,  0.03239167, -0.01391454,  0.02749405, -0.03802495,
         0.01659871, -0.04137981, -0.02636231,  0.03173229, -0.03059345,
         0.01815433,  0.04061231,  0.01482496, -0.01484979, -0.03857679,
        -0.0100625 , -0.02251296, -0.00547212,  0.01178497,  0.00934781,
        -0.00408497, -0.02145313,  0.00995749,  0.00993848,  0.01580167,
        -0.02879154,  0.01962933, -0.015732  , -0.04672234,  0.00841181,
         0.01831106,  0.04613948, -0.04699216, -0.00527031, -0.01492484,
         0.0109151 , -0.04063094, -0.03076116,  0.03779374,  0.03917104,
         0.04117602, -0.01394352,  0.027944  , -0.04491594, -0.00537759,
         0.00360585, -0.04391625, -0.0177028 ,  0.04683701,  0.04770115,
         0.01547693, -0.03204831, -0.04116507, -0.04808186,  0.0209965 ,
         0.01703209, -0.00917921,  0.01772796,  0.04209289,  0.01021303,
         0.01774074,  0.00112623, -0.04534898, -0.03615272,  0.03691976,
  

In [25]:
#012 Discussing the various modelling experiments we're going to run
#013 Model 0 Building a baseline model to try and improve upon
#Multinomial naive bais using TF_IDF formula
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),

])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
#Evalute our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score*100:.2f}%")

Our baseline model achives an accuracy of : 79.27%


In [27]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [28]:
#014 Creating a function to track and evaluate our model's results
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred)*100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true, y_pred,average="weighted")
    model_results = {"accuracy":model_accuracy,
                     "precision":model_precision,
                     "recall":model_recall,
                     "f1":model_f1}
    return model_results

In [29]:
baseline_results = calculate_results(y_true=val_labels,y_pred=baseline_preds)
baseline_results
#higher precision leads to less false positives
#higher recall leads to less false negetives

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [30]:
#015 Model 1 Building, fitting and evaluating our first deep model on text data
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers
SAVE_DIR = "model_logs"
#build model with Function API
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x) #layers.Reshape((1920,))(x) #.
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")
model_1.summary()



Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [31]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240305-060409
Epoch 1/5


2024-03-05 06:04:10.897528: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x56297f8f0e70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-05 06:04:10.897573: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-03-05 06:04:10.903705: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-05 06:04:11.107537: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


215/215 [==============================] - 14s 58ms/step - loss: 0.6134 - accuracy: 0.6949 - val_loss: 0.5385 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4436 - accuracy: 0.8209 - val_loss: 0.4702 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3484 - accuracy: 0.8610 - val_loss: 0.4609 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2856 - accuracy: 0.8913 - val_loss: 0.4611 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2386 - accuracy: 0.9110 - val_loss: 0.4793 - val_accuracy: 0.7861


In [32]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4793 - accuracy: 0.7861


[0.4793362021446228, 0.7860892415046692]

In [33]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0]

24/24 [==============================] - 0s 2ms/step


array([0.38656616], dtype=float32)

In [34]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [35]:
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [36]:
#calculate model 1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7911704681403405,
 'recall': 0.7860892388451444,
 'f1': 0.7830068921982195}

In [37]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [38]:
#016 Visualising our model's learned word embeddings with TensorFlow's projector tool
words_in_vocab = text_vectorizer.get_vocabulary()

In [39]:
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [40]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [41]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape
#https://projector.tensorflow.org/
#https://www.tensorflow.org/text/guide/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk

(10000, 128)

In [42]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [43]:
#017 High-level overview of Recurrent Neural Networks (RNNs) + where to learn more
#018 Model 2 Building, fitting and evaluating our first TensorFlow RNN model (LSTM)
#MOdel 2 LSTM
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")


In [44]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [45]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240305-060432
Epoch 1/5


2024-03-05 06:04:35.691322: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8204


215/215 [==============================] - 12s 35ms/step - loss: 0.2237 - accuracy: 0.9177 - val_loss: 0.5444 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1564 - accuracy: 0.9423 - val_loss: 0.6484 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1289 - accuracy: 0.9505 - val_loss: 0.6484 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1038 - accuracy: 0.9604 - val_loss: 0.8352 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0825 - accuracy: 0.9664 - val_loss: 0.8742 - val_accuracy: 0.7795


In [46]:
#make prediction with LSTM Model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.4178805e-01],
       [6.7764586e-01],
       [9.9971789e-01],
       [5.0810535e-02],
       [7.5773179e-04],
       [9.9932861e-01],
       [5.9049714e-01],
       [9.9979204e-01],
       [9.9966693e-01],
       [2.9446393e-01]], dtype=float32)

In [47]:
model_2_pred=tf.squeeze(tf.round(model_2_pred_probs))

In [48]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_pred)

In [49]:
model_2_results

{'accuracy': 77.95275590551181,
 'precision': 0.7811151206076258,
 'recall': 0.7795275590551181,
 'f1': 0.7776506830100394}

In [50]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [51]:
#019 Model 3 Building, fitting and evaluating a GRU-cell powered RNN
#Gated Recurrent Unit
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x= layers.LSTM(64,return_sequences=True)(x)
# x = layers.GRU(64)(x)
# x= layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [52]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                       

In [53]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240305-060453
Epoch 1/5
215/215 [==============================] - 10s 36ms/step - loss: 0.1632 - accuracy: 0.9337 - val_loss: 0.9653 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0799 - accuracy: 0.9702 - val_loss: 1.0510 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0628 - accuracy: 0.9737 - val_loss: 1.0696 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0543 - accuracy: 0.9756 - val_loss: 1.2664 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0519 - accuracy: 0.9764 - val_loss: 1.4453 - val_accuracy: 0.7572


In [54]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_pred)
model_3_results

24/24 [==============================] - 0s 2ms/step


{'accuracy': 75.7217847769029,
 'precision': 0.7567936195399113,
 'recall': 0.7572178477690289,
 'f1': 0.7567837670446225}

In [55]:
model_2_results

{'accuracy': 77.95275590551181,
 'precision': 0.7811151206076258,
 'recall': 0.7795275590551181,
 'f1': 0.7776506830100394}

In [56]:
#020 Model 4 Building, fitting and evaluating a bidirectional RNN model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x= layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [57]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                             

In [58]:
model_4.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])
            

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240305-060512
Epoch 1/5
215/215 [==============================] - 16s 45ms/step - loss: 0.0996 - accuracy: 0.9648 - val_loss: 0.8867 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0513 - accuracy: 0.9764 - val_loss: 1.2934 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 12ms/step - loss: 0.0473 - accuracy: 0.9780 - val_loss: 1.2819 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0425 - accuracy: 0.9774 - val_loss: 1.5253 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0396 - accuracy: 0.9804 - val_loss: 1.4642 - val_accuracy: 0.7651


In [59]:
model_4_pred_probs = model_4.predict(val_sentences)

24/24 [==============================] - 1s 4ms/step


In [60]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [61]:
model_4_results = calculate_results(y_true=val_labels,y_pred = model_4_preds)

In [62]:
model_4_results

{'accuracy': 76.50918635170603,
 'precision': 0.766006698423534,
 'recall': 0.7650918635170604,
 'f1': 0.7632813381351602}

In [63]:
model_3_results

{'accuracy': 75.7217847769029,
 'precision': 0.7567936195399113,
 'recall': 0.7572178477690289,
 'f1': 0.7567837670446225}

In [64]:
#021 Discussing the intuition behind Conv1D neural networks for text and sequences
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters = 64,
                        kernel_size = 5,
                        activation="relu",
                        padding = "same")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape ,conv_1d_output.shape , max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 64]), TensorShape([1, 64]))

In [65]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01990707, -0.02916985,  0.00346183, ...,  0.03971563,
          0.02587745,  0.02301648],
        [-0.00251488,  0.01644602, -0.00301683, ..., -0.00892   ,
          0.05673584,  0.0037389 ],
        [ 0.04312012,  0.05308859,  0.02159369, ..., -0.02085842,
         -0.04715092,  0.05602638],
        ...,
        [-0.01749046,  0.03103767, -0.0022359 , ..., -0.00039577,
         -0.00679826,  0.01902184],
        [-0.01749046,  0.03103767, -0.0022359 , ..., -0.00039577,
         -0.00679826,  0.01902184],
        [-0.01749046,  0.03103767, -0.0022359 , ..., -0.00039577,
         -0.00679826,  0.01902184]]], dtype=float32)>

In [66]:
max_pool_output

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[0.02639175, 0.08600247, 0.00257779, 0.11316836, 0.0230539 ,
        0.04024326, 0.01896715, 0.00586041, 0.04086346, 0.04274008,
        0.04235326, 0.06791104, 0.03338356, 0.02080641, 0.06215672,
        0.02694988, 0.05080918, 0.02188557, 0.01131164, 0.06782988,
        0.04296222, 0.00871401, 0.07203022, 0.02187985, 0.03143556,
        0.01105652, 0.        , 0.03517973, 0.03346103, 0.0472384 ,
        0.04064376, 0.02034661, 0.060712  , 0.06319491, 0.05412596,
        0.0732749 , 0.0661853 , 0.04887849, 0.03809064, 0.07165462,
        0.02528686, 0.0276967 , 0.1500001 , 0.0748684 , 0.05856758,
        0.0783086 , 0.04932248, 0.01245319, 0.05300895, 0.05350839,
        0.05682465, 0.06075427, 0.03768881, 0.02731232, 0.06461995,
        0.03845749, 0.03512598, 0.05398038, 0.07912284, 0.02354084,
        0.01992917, 0.05249196, 0.0128468 , 0.03349431]], dtype=float32)>

In [67]:
#022 Model 5 Building, fitting and evaluating a 1D CNN for text
from tensorflow.keras import layers
input = layers.Input(shape=(1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,kernel_size=5,strides = 1,activation = "relu",padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name="model_5_Conv1D")

model_5.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [68]:
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [69]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20240305-060545
Epoch 1/5
215/215 [==============================] - 9s 34ms/step - loss: 0.1255 - accuracy: 0.9616 - val_loss: 0.8585 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0761 - accuracy: 0.9715 - val_loss: 1.0417 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0630 - accuracy: 0.9756 - val_loss: 1.1086 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0553 - accuracy: 0.9764 - val_loss: 1.1515 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0524 - accuracy: 0.9781 - val_loss: 1.2018 - val_accuracy: 0.7546


In [70]:
model_5_pred_probs = model_5.predict(val_sentences)

24/24 [==============================] - 0s 3ms/step


In [71]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

In [72]:
model_5_results = calculate_results(y_true = val_labels,
                                    y_pred = model_5_preds,)
model_5_results

{'accuracy': 75.45931758530183,
 'precision': 0.7545478978669297,
 'recall': 0.7545931758530183,
 'f1': 0.7533013461853509}

In [73]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [74]:
#023 Using TensorFlow Hub for pretrained word embeddings (transfer learning for NLP)
#Universal sentence encoder
sample_sentence = "There's a flood in my street!"
import tensorflow_hub as hub
import tensorflow as tf

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")
with tf.device('/CPU:0'):
    embed_samples = embed([sample_sentence, "When you call the universal sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples)


tf.Tensor(
[[-0.01157022  0.02485909  0.02878049 ... -0.00186123  0.0231582
  -0.01485022]
 [ 0.0359669  -0.08579467 -0.01152745 ... -0.03414334  0.02816024
  -0.00878945]], shape=(2, 512), dtype=float32)


In [75]:
#024 Model 6 Building, training and evaluating a transfer learning model for NLP
# create a keras layer using the USE pretrained layer from tensorflow hub
with tf.device('/CPU:0'):
    sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",input_shape = [],dtype = tf.string,trainable=False,name = "USE")

In [76]:
# create model using the sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1,activation="sigmoid")
],name = "model_6_USE")

model_6.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [77]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [78]:
#Train a classifier on top of USE pretrained embeddings
# with tf.device('/CPU:0'):
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"tf_hub_sentence_encoder_USE")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_USE/20240305-060637
Epoch 1/5
215/215 [==============================] - 7s 16ms/step - loss: 0.5071 - accuracy: 0.7835 - val_loss: 0.4465 - val_accuracy: 0.8018
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4144 - accuracy: 0.8168 - val_loss: 0.4417 - val_accuracy: 0.8123
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3995 - accuracy: 0.8213 - val_loss: 0.4357 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3925 - accuracy: 0.8264 - val_loss: 0.4279 - val_accuracy: 0.8150
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3861 - accuracy: 0.8316 - val_loss: 0.4279 - val_accuracy: 0.8150


In [79]:
with tf.device('/CPU:0'):
    model_6_pred_probs = model_6.predict(val_sentences)
    model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
    model_6_results = calculate_results(y_true=val_labels,y_pred=model_6_preds)


24/24 [==============================] - 1s 5ms/step


In [80]:
model_6_results

{'accuracy': 81.49606299212599,
 'precision': 0.8175707797736699,
 'recall': 0.8149606299212598,
 'f1': 0.8133343159175271}

In [81]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [95]:
#025 Preparing subsets of data for model 7 (same as model 6 but 10% of data)
# Model7 : Tf Hub pretrained USE but with 10% of training data
# Transfer Leraning helps when we do not have large dataset
#Below code causes data leakage
# train_10_percent = train_df_shuffled[["text","target"]].sample(frac = 0.1,random_state=42)
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()

# Fixing data leakage
train_10_percent_split = int(0.1*len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]


In [96]:
train_10_percent["target"].value_counts()

0    413
1    348
Name: target, dtype: int64

In [97]:
#026 Model 7 Building, training and evaluating a transfer learning model on 10% data
# model_7 = tf.keras.models.clone_model(model_6)
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1,activation="sigmoid")
],name = "model_7_USE")
model_7.compile(loss ="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_10 (Dense)            (None, 64)                32832     
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [98]:
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"tf_hub_sentence_encoder_USE_10_percent_correct_split")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_USE_10_percent_correct_split/20240305-071550
Epoch 1/5
22/22 [==============================] - 3s 51ms/step - loss: 0.6699 - accuracy: 0.6569 - val_loss: 0.6453 - val_accuracy: 0.7454
Epoch 2/5
22/22 [==============================] - 1s 25ms/step - loss: 0.5967 - accuracy: 0.7956 - val_loss: 0.5838 - val_accuracy: 0.7585
Epoch 3/5
22/22 [==============================] - 1s 24ms/step - loss: 0.5206 - accuracy: 0.8029 - val_loss: 0.5310 - val_accuracy: 0.7730
Epoch 4/5
22/22 [==============================] - 1s 25ms/step - loss: 0.4609 - accuracy: 0.8146 - val_loss: 0.4987 - val_accuracy: 0.7756
Epoch 5/5
22/22 [==============================] - 1s 25ms/step - loss: 0.4203 - accuracy: 0.8248 - val_loss: 0.4877 - val_accuracy: 0.7756


In [99]:
#make Predictions with 10 percent model
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred= model_7_preds)

24/24 [==============================] - 1s 10ms/step


In [100]:
model_7_results

{'accuracy': 77.55905511811024,
 'precision': 0.7772070861555818,
 'recall': 0.7755905511811023,
 'f1': 0.7736182129212565}

In [ ]:
#027 Fixing our data leakage issue with model 7 and retraining it.mp4
#028 Comparing all our modelling experiments evaluation metrics